In [ ]:
# Uncomment and run this cell if you're on Colab or Kaggle
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
# from install import *
# install_requirements(is_chapter2=True)

from install import *
install_requirements(is_chapter2=True)

In [ ]:
from utils import *
setup_chapter()

In [ ]:
from datasets import load_dataset
emotions = load_dataset("dair-ai/emotion")